###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
import pickle
from collections import deque
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'

# American Community Survey 2017 1 Year Estimates

In [2]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [3]:
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']

In [4]:
GNRC = ['111', #Macon
       '161', #Stewart
       '125', #Montgomery
       '083', #Houston
       '085', #Humphreys
       '043', #Dickson
       '021', #Cheatham
       '147', #Robertson
       '165', #Sumner
       '037', #Davidson
       '189', #Wilson
       '169', #Trousdale
       '187', #Williamson
       '149', #Rutherford
       '119'] #Maury
KY = ['003', #Allen
      '213'] #Simpson
qlaces = ['1600000US4702180', #Ashland City: Cheatham
          '1600000US4739660', #Kingston Springs: Cheatham
          '1600000US4757480', #Pegram: Cheatham
          '1600000US4759560', #Pleasant View: Cheatham
          '1600000US4704620', #Belle Meade: Davidson
          '1600000US4705140', #Berry Hill: Davidson
          '1600000US4727020', #Forest Hills: Davidson
          '1600000US4729920', #Goodlettsville: Davidson/Sumner
          '1600000US4752006', #Nashville-Davidson metropolitan government (balance): Davidson
          '1600000US4754780', #Oak Hill: Davidson
          '1600000US4763140', #Ridgetop: Davidson/Robertson
          '1600000US4709880', #Burns: Dickson
          '1600000US4713080', #Charlotte: Dickson
          '1600000US4720620', #Dickson: Dickson
          '1600000US4769080', #Slayden: Dickson
          '1600000US4776860', #Vanleer: Dickson
          '1600000US4779980', #White Bluff: Dickson
          '1600000US4724320', #Erin: Houston
          '1600000US4773460', #Tennessee Ridge: Houston/Stewart
          '1600000US4744840', #McEwen: Humphreys
          '1600000US4752820', #New Johnsonville: Humphreys
          '1600000US4778560', #Waverly: Humphreys
          '1600000US4716540', #Columbia: Maury
          '1600000US4751080', #Mount Pleasant: Maury
          '1600000US4770580', #Spring Hill: Maury/Williamson
          '1600000US4715160', #Clarksville: Montgomery
          '1600000US4700200', #Adams: Robertson
          '1600000US4711980', #Cedar Hill: Robertson
          '1600000US4716980', #Coopertown: Robertson
          '1600000US4718420', #Cross Plains: Robertson
          '1600000US4730960', #Greenbrier: Robertson
          '1600000US4748980', #Millersville: Robertson/Sumner
          '1600000US4760280', #Portland: Robertson/Sumner
          '1600000US4770500', #Springfield: Robertson
          '1600000US4780200', #White House: Robertson/Sumner
          '1600000US4722360', #Eagleville: Rutherford
          '1600000US4741200', #La Vergne: Rutherford
          '1600000US4751560', #Murfreesboro: Rutherford
          '1600000US4769420', #Smyrna: Rutherford
          '1600000US4718820', #Cumberland City: Stewart
          '1600000US4721400', #Dover: Stewart
          '1600000US4728540', #Gallatin: Sumner
          '1600000US4733280', #Hendersonville: Sumner
          '1600000US4779420', #Westmoreland: Sumner
          '1600000US4708280', #Brentwood: Williamson
          '1600000US4725440', #Fairview: Williamson
          '1600000US4727740', #Franklin: Williamson
          '1600000US4753460', #Nolensville: Williamson
          '1600000US4773900', #Thompson's Station: Williamson
          '1600000US4741520', #Lebanon: Wilson
          '1600000US4750780', #Mount Juliet: Wilson
          '1600000US4778320', #Watertown: Wilson
          '1600000US4740160', #Lafayette: Macon
          '1600000US2169114', #Scottsville city, Kentucky
          '1600000US2128918', #Franklin city, Kentucky
          '1600000US4749460'] #Mitchelville city: Sumner

In [5]:
def percent(x, y):
    return (x/y)*100

## Read In Data Guide

In [6]:
dataguide = pd.read_csv('../../Data Guides/DATA GUIDE ACS1YR.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [7]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
#dg2 = dataguide[dataguide['ID'].between(47, 92)]

In [8]:
url_str= 'https://api.census.gov/data/2017/acs/acs1?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'

In [9]:
dataguide = dg1
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [10]:
data = savename
data = data.set_index(['NAME', 'GEO_ID'])
data.head(2)

,,geo_total_series,geo_samehouse_total,geo_movedsamecounty_total,geo_moveddifferentcountysamestate_total,geo_moveddifferentstate_total,geo_movedabroad_total,StateFIPS,GeoFIPS
NAME,GEO_ID,,,,,,,,
"Maury County, Tennessee",0500000US47119,None,None,None,None,None,None,47,119
"Robertson County, Tennessee",0500000US47147,None,None,None,None,None,None,47,147


In [11]:
transp = data.transpose()
transp.head(10)

NAME,"Maury County, Tennessee","Robertson County, Tennessee","Sumner County, Tennessee","Williamson County, Tennessee","Davidson County, Tennessee","Montgomery County, Tennessee","Rutherford County, Tennessee","Wilson County, Tennessee","Franklin city, Tennessee","Murfreesboro city, Tennessee","Clarksville city, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee",Tennessee,United States
GEO_ID,0500000US47119,0500000US47147,0500000US47165,0500000US47187,0500000US47037,0500000US47125,0500000US47149,0500000US47189,1600000US4727740,1600000US4751560,1600000US4715160,1600000US4752006,0400000US47,0100000US
geo_total_series,None,None,181562,223906,680507,195777,312925,None,None,134814,149422,655387,6642072,321994428
geo_samehouse_total,None,None,153887,188492,551997,145895,265690,None,None,106846,108084,530723,5675990,275921008
geo_movedsamecounty_total,None,None,13703,9044,76528,22840,20429,None,None,10865,17419,74902,541230,25960510
geo_moveddifferentcountysamestate_total,None,None,7551,15925,16492,4073,17054,None,None,11028,3408,15278,207682,10508089
geo_moveddifferentstate_total,None,None,5816,10050,30785,20447,8791,None,None,5281,18644,29779,191909,7471493
geo_movedabroad_total,None,None,605,395,4705,2522,961,None,None,794,1867,4705,25261,2133328
StateFIPS,47,47,47,47,47,47,47,47,47,47,47,47,47,1
GeoFIPS,119,147,165,187,037,125,149,189,27740,51560,15160,52006,0,0


In [12]:
transp = transp.drop(columns = [('Maury County, Tennessee', '0500000US47119'), ('Robertson County, Tennessee', '0500000US47147'),
                                ('Wilson County, Tennessee', '0500000US47189'), ('Franklin city, Tennessee', '1600000US4727740')])

In [13]:
numcols = list(transp.columns)
numcols
transp[numcols] = transp[numcols].astype(float)

In [14]:
data = transp

In [15]:
data.head()

NAME,"Sumner County, Tennessee","Williamson County, Tennessee","Davidson County, Tennessee","Montgomery County, Tennessee","Rutherford County, Tennessee","Murfreesboro city, Tennessee","Clarksville city, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee",Tennessee,United States
GEO_ID,0500000US47165,0500000US47187,0500000US47037,0500000US47125,0500000US47149,1600000US4751560,1600000US4715160,1600000US4752006,0400000US47,0100000US
geo_total_series,181562.0,223906.0,680507.0,195777.0,312925.0,134814.0,149422.0,655387.0,6642072.0,321994428.0
geo_samehouse_total,153887.0,188492.0,551997.0,145895.0,265690.0,106846.0,108084.0,530723.0,5675990.0,275921008.0
geo_movedsamecounty_total,13703.0,9044.0,76528.0,22840.0,20429.0,10865.0,17419.0,74902.0,541230.0,25960510.0
geo_moveddifferentcountysamestate_total,7551.0,15925.0,16492.0,4073.0,17054.0,11028.0,3408.0,15278.0,207682.0,10508089.0
geo_moveddifferentstate_total,5816.0,10050.0,30785.0,20447.0,8791.0,5281.0,18644.0,29779.0,191909.0,7471493.0


### Not aggregating for regions or unincorporated areas as data is only available for select geographies.

In [16]:
data = data.transpose()
coldropper = data
data = data.reset_index()

In [17]:
#calculations
#geographic mobility
data['GeoMobility:Total'] = data['geo_total_series']
data['GeoMobility:Same House'] = data['geo_samehouse_total']
data['GeoMobility%:Same House'] = percent(data['GeoMobility:Same House'],data['GeoMobility:Total'])
data['GeoMobility:Moved Same County'] = data['geo_movedsamecounty_total']
data['GeoMobility%:Moved Same County'] = percent(data['GeoMobility:Moved Same County'],data['GeoMobility:Total'])
data['GeoMobility:Moved Different County Same State'] = data['geo_moveddifferentcountysamestate_total']
data['GeoMobility%:Moved Different County Same State'] = percent(data['GeoMobility:Moved Different County Same State'],data['GeoMobility:Total'])
data['GeoMobility:Moved Different State'] = data['geo_moveddifferentstate_total']
data['GeoMobility%:Moved Different State'] = percent(data['GeoMobility:Moved Different State'],data['GeoMobility:Total'])
data['GeoMobility:Moved From Abroad'] = data['geo_movedabroad_total']
data['GeoMobility%:Moved From Abroad'] = percent(data['GeoMobility:Moved From Abroad'],data['GeoMobility:Total'])

In [18]:
cols = coldropper.columns
data = data.drop(columns = cols)
data.head(3)

,NAME,GEO_ID,GeoMobility:Total,GeoMobility:Same House,GeoMobility%:Same House,GeoMobility:Moved Same County,GeoMobility%:Moved Same County,GeoMobility:Moved Different County Same State,GeoMobility%:Moved Different County Same State,GeoMobility:Moved Different State,GeoMobility%:Moved Different State,GeoMobility:Moved From Abroad,GeoMobility%:Moved From Abroad
0,"Sumner County, Tennessee",0500000US47165,181562.0,153887.0,84.757273,13703.0,7.547284,7551.0,4.158910,5816.0,3.203313,605.0,0.333220
1,"Williamson County, Tennessee",0500000US47187,223906.0,188492.0,84.183541,9044.0,4.039195,15925.0,7.112360,10050.0,4.488491,395.0,0.176413
2,"Davidson County, Tennessee",0500000US47037,680507.0,551997.0,81.115551,76528.0,11.245733,16492.0,2.423487,30785.0,4.523833,4705.0,0.691396


In [19]:
data.to_csv('../../Outputs/2017ACS1YR.csv', index = False)